In [ ]:
import torch

In [ ]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, ViTModel, ViTFeatureExtractor
import random
import numpy as np

##FLIP 모델 클래스

In [ ]:
class FLIPModel(nn.Module):
    def __init__(self, image_masking_ratio=0.75):
        super().__init__()
        # 텍스트 인코더 (BERT 기반)
        self.text_encoder = BertModel.from_pretrained('bert-base-uncased')
        # 이미지 인코더 (ViT 기반)
        self.image_encoder = ViTModel.from_pretrained('google/vit-base-patch16-224-in21k')

        # 프로젝션 레이어: 공통된 임베딩 공간으로 매핑
        self.text_projection = nn.Linear(self.text_encoder.config.hidden_size, 512)
        self.image_projection = nn.Linear(self.image_encoder.config.hidden_size, 512)

        # 이미지 마스킹 비율
        self.image_masking_ratio = image_masking_ratio

        # 학습 가능한 마스크 토큰 (대체 토큰)
        self.mask_token = nn.Parameter(torch.zeros(self.image_encoder.config.hidden_size))
        nn.init.normal_(self.mask_token, std=0.02)  # 표준 정규 분포로 초기화

    def encode_image(self, pixel_values, mask=False):
        """
        이미지 인코딩 - 마스킹 옵션 포함
        """
        if not mask:
            # 마스킹 없이 일반 인코딩
            image_outputs = self.image_encoder(pixel_values=pixel_values)
            image_embeddings = image_outputs.last_hidden_state[:, 0]  # [CLS] 토큰 사용
            image_embeddings = self.image_projection(image_embeddings)
            return image_embeddings
        else:
            # 마스킹 적용
            masked_embeddings = self.apply_image_masking(pixel_values)
            masked_cls_embedding = masked_embeddings[:, 0]  # [CLS] 토큰
            masked_image_embeddings = self.image_projection(masked_cls_embedding)
            return masked_image_embeddings

    def encode_text(self, input_ids, attention_mask, token_type_ids=None):
        """텍스트 인코딩"""
        text_outputs = self.text_encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            return_dict=True
        )
        text_embeddings = text_outputs.last_hidden_state[:, 0]  # [CLS] 토큰 사용
        text_embeddings = self.text_projection(text_embeddings)
        return text_embeddings

    def apply_image_masking(self, pixel_values):
        """이미지에 마스킹 적용 (FLIP의 핵심 메커니즘)"""
        batch_size = pixel_values.size(0)
        device = pixel_values.device

        # ViT는 이미지를 패치로 분할하여 처리
        # 먼저 이미지를 인코더에 전달하여 패치 임베딩 획득
        image_outputs = self.image_encoder(pixel_values=pixel_values, output_hidden_states=True)
        patch_embeddings = image_outputs.last_hidden_state  # [batch_size, seq_len, hidden_dim]

        # 첫 번째 토큰은 [CLS] 토큰이므로 제외
        num_patches = patch_embeddings.size(1) - 1  # -1 for CLS token

        # 마스킹할 패치 수 결정
        num_mask = int(self.image_masking_ratio * num_patches)

        # 각 배치 항목에 대해 개별적으로 마스킹 적용
        masked_embeddings = []

        for b in range(batch_size):
            # 마스킹할 패치 인덱스 랜덤 선택 (첫 번째 [CLS] 토큰 제외)
            mask_indices = torch.tensor(random.sample(range(1, num_patches + 1), num_mask), device=device)

            # 마스킹된 임베딩 생성 (마스크 토큰으로 대체)
            curr_embeddings = patch_embeddings[b].clone()
            curr_embeddings[mask_indices] = self.mask_token

            masked_embeddings.append(curr_embeddings)

        # 배치 차원으로 다시 결합
        masked_embeddings = torch.stack(masked_embeddings)

        # 추가 옵션: 마스킹된 패치 임베딩을 다시 인코더를 통과시킬 수 있음
        # masked_embeddings = self.image_encoder.encoder(masked_embeddings)[0]

        return masked_embeddings

    def forward(self, pixel_values, input_ids, attention_mask, token_type_ids=None):
        """FLIP 모델의 포워드 패스"""
        # 이미지 마스킹 적용하고 인코딩
        masked_image_embeddings = self.encode_image(pixel_values, mask=True)

        # 텍스트 인코딩
        text_embeddings = self.encode_text(
            input_ids=input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )

        # 대조 손실(Contrastive Loss)을 위한 정규화
        image_embeddings = F.normalize(masked_image_embeddings, p=2, dim=-1)
        text_embeddings = F.normalize(text_embeddings, p=2, dim=-1)

        # 이미지-텍스트 매칭을 위한 코사인 유사도 계산
        logits_per_image = torch.matmul(image_embeddings, text_embeddings.t())
        logits_per_text = logits_per_image.t()

        return {
            "logits_per_image": logits_per_image,
            "logits_per_text": logits_per_text
        }

In [ ]:
# FLIP의 손실 함수 (대조 손실만 사용)
class FLIPLoss(nn.Module):
    def __init__(self, temperature=0.07):
        super().__init__()
        self.temperature = temperature
        self.cross_entropy = nn.CrossEntropyLoss()

    def forward(self, outputs, batch_size):
        # 대조 손실 계산 (CLIP 스타일)
        logits_per_image = outputs["logits_per_image"] / self.temperature
        logits_per_text = outputs["logits_per_text"] / self.temperature

        # 대각선 요소가 정답 (각 이미지는 해당 텍스트와 매칭)
        labels = torch.arange(batch_size, device=logits_per_image.device)
        loss_img = self.cross_entropy(logits_per_image, labels)
        loss_txt = self.cross_entropy(logits_per_text, labels)

        # 양방향 대조 손실의 평균
        contrastive_loss = (loss_img + loss_txt) / 2

        return contrastive_loss


In [ ]:
# 학습 데이터셋 예시
class ImageTextDataset(Dataset):
    def __init__(self, image_paths, captions, image_processor, tokenizer, max_length=77):
        self.image_paths = image_paths
        self.captions = captions
        self.image_processor = image_processor
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image = self.load_image(self.image_paths[idx])
        caption = self.captions[idx]

        # 이미지 전처리
        pixel_values = self.image_processor(image, return_tensors="pt").pixel_values.squeeze(0)

        # 텍스트 토큰화
        encoded_text = self.tokenizer(
            caption,
            padding="max_length",
            max_length=self.max_length,
            truncation=True,
            return_tensors="pt"
        )

        return {
            "pixel_values": pixel_values,
            "input_ids": encoded_text.input_ids.squeeze(0),
            "attention_mask": encoded_text.attention_mask.squeeze(0),
            "token_type_ids": encoded_text.token_type_ids.squeeze(0) if hasattr(encoded_text, "token_type_ids") else None
        }

    def load_image(self, image_path):
        # 실제 구현에서는 PIL.Image.open(image_path) 등으로 이미지 로드
        # 간단한 예시를 위해 더미 데이터 반환
        return {"dummy_image": True}

In [ ]:
# 학습 루프 예시
def train_flip_model(model, dataloader, optimizer, loss_fn, epochs=1):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    for epoch in range(epochs):
        model.train()
        total_loss = 0

        for batch in dataloader:
            # 배치 데이터를 디바이스로 이동
            pixel_values = batch["pixel_values"].to(device)
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            token_type_ids = batch["token_type_ids"].to(device) if batch.get("token_type_ids") is not None else None

            # 포워드 패스
            outputs = model(
                pixel_values=pixel_values,
                input_ids=input_ids,
                attention_mask=attention_mask,
                token_type_ids=token_type_ids
            )

            # 손실 계산
            batch_size = pixel_values.size(0)
            loss = loss_fn(outputs, batch_size)

            # 역전파 및 옵티마이저 스텝
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

        avg_loss = total_loss / len(dataloader)
        print(f"Epoch {epoch+1}, Loss: {avg_loss:.4f}")


In [ ]:
# 모델 초기화 및 학습 설정 예시
def initialize_and_train():
    # 초기화
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    image_processor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')
    model = FLIPModel(image_masking_ratio=0.75)

    # 옵티마이저 설정
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

    # 손실 함수 설정
    loss_fn = FLIPLoss(temperature=0.07)

    # 데이터셋 및 데이터로더 설정 (실제 구현시에는 실제 데이터 필요)
    # 간단한 예시를 위한 더미 데이터
    image_paths = ["img1.jpg", "img2.jpg", "img3.jpg"]
    captions = ["a dog running", "sunset over mountains", "city at night"]

    dataset = ImageTextDataset(
        image_paths=image_paths,
        captions=captions,
        image_processor=image_processor,
        tokenizer=tokenizer
    )

    dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

    # 학습 실행
    train_flip_model(model, dataloader, optimizer, loss_fn, epochs=3)

    return model

In [ ]:
# FLIP 추론 예시
def flip_inference(model, image, text_candidates, image_processor, tokenizer):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval()

    # 이미지 전처리
    pixel_values = image_processor(image, return_tensors="pt").pixel_values.to(device)

    # 텍스트 후보 토큰화
    encoded_texts = []
    for text in text_candidates:
        encoded_text = tokenizer(
            text,
            padding="max_length",
            max_length=77,
            truncation=True,
            return_tensors="pt"
        )
        encoded_texts.append({
            "input_ids": encoded_text.input_ids.to(device),
            "attention_mask": encoded_text.attention_mask.to(device),
            "token_type_ids": encoded_text.token_type_ids.to(device) if hasattr(encoded_text, "token_type_ids") else None
        })

    with torch.no_grad():
        # 이미지 인코딩 (마스킹 없이 - 추론 시에는 마스킹 사용 안 함)
        image_embeddings = model.encode_image(pixel_values, mask=False)
        image_embeddings = F.normalize(image_embeddings, p=2, dim=-1)

        # 각 텍스트 후보에 대한 유사도 계산
        similarities = []
        for encoded_text in encoded_texts:
            text_embeddings = model.encode_text(
                input_ids=encoded_text["input_ids"],
                attention_mask=encoded_text["attention_mask"],
                token_type_ids=encoded_text["token_type_ids"]
            )
            text_embeddings = F.normalize(text_embeddings, p=2, dim=-1)

            # 유사도 계산
            similarity = torch.matmul(image_embeddings, text_embeddings.t()).item()
            similarities.append(similarity)

    # 가장 높은 유사도를 가진 텍스트 반환
    best_match_idx = np.argmax(similarities)
    return text_candidates[best_match_idx], similarities

In [ ]:
# 메인 실행
if __name__ == "__main__":
    model = initialize_and_train()

    # 모델 저장
    torch.save(model.state_dict(), "flip_model.pt")
    print("모델 학습 및 저장 완료!")

    # 추론 예시
    # 실제 구현 시에는 실제 이미지와 후보 텍스트 필요
    dummy_image = {"dummy_image": True}
    text_candidates = ["a dog running", "sunset over mountains", "city at night"]
    tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
    image_processor = ViTFeatureExtractor.from_pretrained('google/vit-base-patch16-224-in21k')

    best_match, similarities = flip_inference(model, dummy_image, text_candidates, image_processor, tokenizer)
    print(f"최적 매칭 텍스트: {best_match}")
    print(f"유사도 점수: {similarities}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

...

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:28: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

ValueError: Invalid image type. Expected either PIL.Image.Image, numpy.ndarray, torch.Tensor, tf.Tensor or jax.ndarray, but got <class 'dict'>.